Homework

The goal of this homework is to train a simple model for predicting the duration of a ride - similar to what we did in this module.

In [1]:
!wget -P ./data https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet -q

In [2]:
!wget -P ./data https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet -q

Q1. Downloading the data

We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2022.

Read the data for January. How many columns are there?

- 16
- 17
- 18
- **19**

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error


In [4]:
def data_import(path):    
    df = pd.read_parquet(path) 
    return df

In [5]:
path_jan = "./data/yellow_tripdata_2022-01.parquet"
path_feb = "./data/yellow_tripdata_2022-02.parquet"

In [6]:
df_jan = data_import(path_jan)
df_jan.info()
print(f"\nNo. of columns: {len(df_jan.columns)}\n")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2463931 entries, 0 to 2463930
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

Q2. Computing duration

Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

- 41.45
- **46.45**
- 51.45
- 56.45

In [7]:
def compute_duration_mins(df):
    df["duration"] = (df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"])
    df["duration"] = df.duration.apply(lambda x: x.total_seconds()/60)
    return df

In [8]:
df_jan = compute_duration_mins(df_jan)

In [9]:
df_jan["duration"].std()

46.44530513776802

Q3. Dropping outliers

Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

- 90%
- 92%
- 95%
- **98%**

In [10]:
def drop_outliers(df):
    df = df[df['duration'].between(1, 60, inclusive='both')]
    return df  

In [11]:
df_jan_drop_outliers = drop_outliers(df_jan)

In [12]:
(df_jan_drop_outliers.shape[0]/df_jan.shape[0])*100

98.27547930522405

Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

    Turn the dataframe into a list of dictionaries
    Fit a dictionary vectorizer
    Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

- 2
- 155
- 345
- **515**
- 715

In [13]:
def select_transform_pickup_dropoff(df):
    df_pickup_dropoff = df[["PULocationID","DOLocationID"]].astype('str')
    return df_pickup_dropoff

In [14]:
def one_hot_encode_train_data(df):

    df_dicts = df.to_dict(orient='records')
    dv = DictVectorizer()
    df_dv = dv.fit_transform(df_dicts)
    return df_dv, dv

In [15]:
df_jan_pickup_dropoff = select_transform_pickup_dropoff(df_jan_drop_outliers)

In [16]:
x_train, dv = one_hot_encode_train_data(df_jan_pickup_dropoff)

In [17]:
x_train.shape

(2421440, 515)

In [18]:
y_train = df_jan_drop_outliers["duration"].values

In [19]:
y_train.shape

(2421440,)

Q5. Training a model

Now let's use the feature matrix from the previous step to train a model.

    Train a plain linear regression model with default parameters
    Calculate the RMSE of the model on the training data

What's the RMSE on train?

- **6.99**
- 11.99
- 16.99
- 21.99

In [20]:
lr = LinearRegression()
model = lr.fit(x_train, y_train)


In [21]:
model.score(x_train, y_train) 

0.39734969363279327

In [22]:
y_train_pred = model.predict(x_train)

In [23]:
rmse = mean_squared_error(y_train_pred, y_train, squared=False)
print(f"RMSE: {rmse}") 

RMSE: 6.986190837370544


Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2022).

What's the RMSE on validation?

- **7.79**
- 12.79
- 17.79
- 22.79

In [24]:
df_feb = data_import(path_feb)

In [25]:
df_feb = compute_duration_mins(df_feb)

In [26]:
df_feb_drop_outliers = drop_outliers(df_feb)

In [27]:
y_val = df_feb_drop_outliers["duration"].values

In [28]:
df_val_pickup_dropoff = select_transform_pickup_dropoff(df_feb_drop_outliers)

In [29]:
df_val_dicts = df_val_pickup_dropoff.to_dict(orient='records')

In [30]:
x_val = dv.transform(df_val_dicts)                        

In [31]:
y_val_pred = model.predict(x_val) 

In [32]:
rmse = mean_squared_error(y_val_pred, y_val, squared=False)
print(f"RMSE: {rmse}") 
                                         

RMSE: 7.786409085078911


In [33]:
model.score(x_val, y_val) 

0.3480092704546819

Submit the results

    Submit your results here: https://forms.gle/uYTnWrcsubi2gdGV7
    You can submit your solution multiple times. In this case, only the last submission will be used
    If your answer doesn't match options exactly, select the closest one

Deadline

The deadline for submitting is 23 May 2023 (Tuesday), 23:00 CEST (Berlin time).

After that, the form will be closed.